##![LearnAI Header](https://coursematerial.blob.core.windows.net/assets/LearnAI_header.png)

# Machine Learning Experimentation and Model Management with AML Services

In this lab you will learn how to integrate Azure Databricks with Azure Machine Learning. 

This powerful combination allows you to do the following:
- Keep track of the performance of your various machine learning solutions
- Register the a candidate model for deployment
- Create a docker image
- Deploy your solution as a webservice

We now run a training experiment and use the Azure ML SDK to save it to our AML Workspace.

In [4]:
import os
import pprint
import numpy as np

from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [5]:
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.17

Let's load our Azure ML Workspace first:

In [7]:
# import the Workspace class and check the azureml SDK version
from azureml.core import Workspace

config_path = '/dbfs/tmp/'

ws = Workspace.from_config(path=os.path.join(config_path, 'aml_config','config.json'))
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Found the config file in: /dbfs/tmp/aml_config/config.json
Workspace name: myADBworkspace
Azure region: westus2
Resource group: adb_airlift_rg

## Read the data

In [9]:
df = spark.read.parquet("dbfs:/FileStore/tables/preprocessed").withColumnRenamed("y_0", "label").cache()
display(df)

machineID,datetime,age,diff_error_0,diff_error_1,diff_error_2,diff_error_3,diff_error_4,diff_fail_0,diff_fail_1,diff_fail_2,diff_fail_3,diff_maint_0,diff_maint_1,diff_maint_2,diff_maint_3,pressure_ma_3,pressure_sd_3,rotate_ma_3,rotate_sd_3,vibration_ma_3,vibration_sd_3,volt_ma_3,volt_sd_3,label,y_1,y_2,y_3
32,2015-02-23T21:00:00.000+0000,15,1095.0,166.0,409.0,1387.0,375.0,1071.0,1387.0,1387.0,351.0,711.0,1387.0,1387.0,351.0,99.47241720086753,4.027627389831299,451.38980736669623,37.143235255898006,36.437978246330076,5.246946040863718,175.77508414243,27.49122867502091,0,0,0,0
32,2015-02-23T22:00:00.000+0000,15,1096.0,167.0,410.0,1388.0,376.0,1072.0,1388.0,1388.0,352.0,712.0,1388.0,1388.0,352.0,100.09894109745454,3.794141420379903,435.17321536069,39.335699599244414,39.2308973568302,4.176148610708063,180.83567404661,23.740298384283044,0,0,0,0
32,2015-02-23T23:00:00.000+0000,15,1097.0,168.0,411.0,1389.0,377.0,1073.0,1389.0,1389.0,353.0,713.0,1389.0,1389.0,353.0,99.54439890229081,3.4758830167295156,421.66272855994526,15.279105023168727,41.711212313841905,0.9869915716793245,169.48155636960098,21.844321498131357,0,0,0,0
32,2015-02-24T00:00:00.000+0000,15,1098.0,169.0,412.0,1390.0,378.0,1074.0,1390.0,1390.0,354.0,714.0,1390.0,1390.0,354.0,97.68908923167595,2.826179881000501,416.56100286652577,14.313070539126413,43.106649846394575,2.4962971618940304,168.40722316791698,19.978049808200037,0,0,0,0
32,2015-02-24T01:00:00.000+0000,15,1099.0,170.0,413.0,1391.0,379.0,1075.0,1391.0,1391.0,355.0,715.0,1391.0,1391.0,355.0,96.91916094721387,4.036157929664655,428.15124045533855,23.48933395357947,42.8987242528714,2.771261929609087,172.9402295425665,16.124035462907294,0,0,0,0
32,2015-02-24T02:00:00.000+0000,15,1100.0,171.0,414.0,1392.0,380.0,1076.0,1392.0,1392.0,356.0,716.0,1392.0,1392.0,356.0,98.60820305322397,6.429119080028224,444.72279625965575,26.725216590298036,43.55483208223855,2.8184999168524905,166.95757356868276,19.57722439213211,0,0,0,0
32,2015-02-24T03:00:00.000+0000,15,1101.0,172.0,415.0,1393.0,381.0,1077.0,1393.0,1393.0,357.0,717.0,1393.0,1393.0,357.0,97.75829917013185,6.35926449953383,455.000717946228,30.802700661866158,46.0032332920517,5.284176271790364,171.5146198226785,18.02024867702125,0,0,0,0
32,2015-02-24T04:00:00.000+0000,15,1102.0,173.0,416.0,1394.0,382.0,1078.0,1394.0,1394.0,358.0,718.0,1394.0,1394.0,358.0,97.46746997114926,6.507590246654807,464.63403314151725,13.781545445572954,44.00402649786548,6.376598773584249,163.08366794565802,10.130385844311215,0,0,0,0
32,2015-02-24T05:00:00.000+0000,15,1103.0,174.0,417.0,1395.0,383.0,1079.0,1395.0,1395.0,359.0,719.0,1395.0,1395.0,359.0,103.47412855441392,9.726682380322783,481.54989358335945,31.29419249089553,41.575592514816776,9.504942586795327,164.627163319354,11.547022705888775,0,0,0,0
32,2015-02-24T06:00:00.000+0000,15,1104.0,175.0,418.0,1396.0,384.0,1080.0,1396.0,1396.0,360.0,720.0,0.0,1396.0,360.0,99.97419460128005,10.652800910349333,479.901478683917,32.041360774820994,39.010816091632826,9.76504784353832,168.14560647024325,6.929237687861469,0,0,0,0


In [10]:
keys = ['machineID', 'datetime']
X_keep = ['diff_maint_1', 'diff_error_1', 'volt_sd_3', 'diff_fail_3', 'pressure_ma_3', 'pressure_sd_3', 'diff_fail_1', 'diff_fail_0', 'age', 'vibration_ma_3', 'rotate_ma_3', 'diff_error_2', 'diff_fail_2', 'diff_error_3', 'diff_maint_2', 'volt_ma_3', 'diff_maint_0', 'vibration_sd_3', 'diff_maint_3', 'rotate_sd_3', 'diff_error_0', 'diff_error_4']
Y_keep = ['y_0', 'y_1', 'y_2', 'y_3']

In [11]:
# from pyspark.sql.types import DateType
from pandas import datetime
from pyspark.sql.functions import col, hour

# we sample every nth row of the data using the `hour` function
df_train = df.filter((col('datetime') < datetime(2015, 10, 1))) # & (hour(col('datetime')) % 3 == 0))
df_test = df.filter(col('datetime') > datetime(2015, 10, 15))

#Define Model

It is time to run the experiment. To do so we load the root experiment and call the `start_logging` method. We then invoke each iteration of the experiment using the `run` and tell it which metrics to log. Examine the code below and see it all happening in action.

In [14]:
from azureml.core.run import Run
from azureml.core.experiment import Experiment
import numpy as np
import os
import shutil
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml import Pipeline

vassembler = VectorAssembler(inputCols = X_keep, outputCol = "features")
stndscaler = StandardScaler(inputCol = "features", outputCol = "norm_features")

model_name = "PdM_logistic_regression.mml"
model_dbfs = os.path.join("/dbfs", model_name)
run_history_name = 'spark-ml-notebook'

# start a training run by defining an experiment
myexperiment = Experiment(ws, "AI_Airlft")
root_run = myexperiment.start_logging()

# Regularization Rates - 
regs = [0.0001, 0.001, 0.01, 0.1]
 
# try a bunch of regularization rate in a Logistic Regression model
for reg in regs:
    print("Regularization rate: {}".format(reg))
    # create a bunch of child runs
    with root_run.child_run("reg-" + str(reg)) as run:
        # create a new Logistic Regression model.
        lr = (LogisticRegression(regParam=reg)
              .setLabelCol("label")
              .setFeaturesCol("norm_features"))
        
        # put together the pipeline
        pipe = Pipeline(stages=[vassembler, stndscaler, lr])

        # train the model
        model_p = pipe.fit(df_train)
        
        # make prediction
        pred = model_p.transform(df_test)
        
        # evaluate. note only 2 metrics are supported out of the box by Spark ML.
        bce = (BinaryClassificationEvaluator()
               .setLabelCol("label")
               .setRawPredictionCol('rawPrediction'))
               
        au_roc = bce.setMetricName('areaUnderROC').evaluate(pred)
        au_prc = bce.setMetricName('areaUnderPR').evaluate(pred)

        print("Area under ROC: {}".format(au_roc))
        print("Area Under PR: {}".format(au_prc))
      
        # log reg, au_roc, au_prc and feature names in run history
        run.log("reg", reg)
        run.log("au_roc", au_roc)
        run.log("au_prc", au_prc)
        run.log_list("columns", df_train.columns)

        # save model
        model_p.write().overwrite().save(model_name)
        
        # upload the serialized model into run history record
        mdl, ext = model_name.split(".")
        model_zip = mdl + ".zip"
        shutil.make_archive(mdl, 'zip', model_dbfs)
        run.upload_file("outputs/" + model_name, model_zip)        
        # run.upload_file("outputs/" + model_name, path_or_stream = model_dbfs) #cannot deal with folders

        # now delete the serialized model from local folder since it is already uploaded to run history 
        shutil.rmtree(model_dbfs)
        os.remove(model_zip)
        
# Declare run completed
root_run.complete()
root_run_id = root_run.id
print ("run id:", root_run.id)

Regularization rate: 0.0001
Area under ROC: 0.8213779036225346
Area Under PR: 0.13206830601445915
Regularization rate: 0.001
Area under ROC: 0.8193678906728926
Area Under PR: 0.13994671526785152
Regularization rate: 0.01
Area under ROC: 0.8222438135578131
Area Under PR: 0.15275686528099713
Regularization rate: 0.1
Area under ROC: 0.8226049714805791
Area Under PR: 0.15462442445185484
run id: a1f553f9-cd20-42c4-b8b8-e3447430b176

In [15]:
# load all run metrics from run history into a dictionary object
child_runs = {}

for r in root_run.get_children():
    child_runs[r.id] = r

In [16]:
child_runs

Out[ 10 ]: 
{'5a99b5f2-9b36-44ff-a717-be6ab79acfbc': Run(Experiment: AI_Airlft,
 Id: 5a99b5f2-9b36-44ff-a717-be6ab79acfbc,
 Type: None,
 Status: Completed),
 '37121d8c-486a-43e5-890b-a9aca114f639': Run(Experiment: AI_Airlft,
 Id: 37121d8c-486a-43e5-890b-a9aca114f639,
 Type: None,
 Status: Completed),
 '31635c3e-365f-48d1-8909-c7b2fbf1d2c8': Run(Experiment: AI_Airlft,
 Id: 31635c3e-365f-48d1-8909-c7b2fbf1d2c8,
 Type: None,
 Status: Completed),
 '3b618e56-2a07-47f4-8723-975df0279259': Run(Experiment: AI_Airlft,
 Id: 3b618e56-2a07-47f4-8723-975df0279259,
 Type: None,
 Status: Completed)}

We can now select the best model based on the metric we choose.

In [18]:
metrics = root_run.get_metrics(recursive = True)
best_run_id = max(metrics, key = lambda k: metrics[k]['au_roc'])
best_run = child_runs[best_run_id]
print('Best run is:', best_run_id)
print('Metrics:', metrics[best_run_id]['au_roc'], metrics[best_run_id]['reg'])

Best run is: 31635c3e-365f-48d1-8909-c7b2fbf1d2c8
Metrics: 0.8226049714805791 0.1

We save the best model on disk for future use.

In [20]:
# download the model from the best run to a local folder
best_model_file_name = "best_model.zip"
best_run.download_file(name = 'outputs/' + model_name, output_file_path = best_model_file_name)

#Model Evaluation

We can load the best model we selected earlier and use it to evaluate its accuracy.

In [23]:
## unzip the model to dbfs (as load() seems to require that) and load it
if os.path.isfile(model_dbfs) or os.path.isdir(model_dbfs):
    shutil.rmtree(model_dbfs)
shutil.unpack_archive(best_model_file_name, model_dbfs)

model_p_best = PipelineModel.load(model_name)

In [24]:
# make prediction
df_pred = model_p_best.transform(df_test)
display(df_pred.groupBy("prediction", "label").count())

prediction,label,count
0.0,0,185712
0.0,1,2088


In [25]:
import pyspark.sql.functions as F

df_select = df_pred.orderBy(F.desc('prediction')).limit(5)
df_select = df_pred.union(df_pred.orderBy(F.asc('prediction')).limit(5))

display(df_select)

machineID,datetime,age,diff_error_0,diff_error_1,diff_error_2,diff_error_3,diff_error_4,diff_fail_0,diff_fail_1,diff_fail_2,diff_fail_3,diff_maint_0,diff_maint_1,diff_maint_2,diff_maint_3,pressure_ma_3,pressure_sd_3,rotate_ma_3,rotate_sd_3,vibration_ma_3,vibration_sd_3,volt_ma_3,volt_sd_3,label,y_1,y_2,y_3,features,norm_features,rawPrediction,probability,prediction
39,2015-10-15T01:00:00.000+0000,0,1759.0,278.0,67.0,231.0,153.0,1843.0,43.0,6983.0,6983.0,1123.0,43.0,1483.0,763.0,94.21543486894998,6.441336953515932,427.1108015695812,52.30227887677106,39.27557152618075,4.513392438930502,166.33543104143877,14.064982347527296,0,0,0,0,"List(1, 22, List(), List(43.0, 278.0, 14.064982347527296, 6983.0, 94.21543486894998, 6.441336953515932, 43.0, 1843.0, 0.0, 39.27557152618075, 427.1108015695812, 67.0, 6983.0, 231.0, 1483.0, 166.33543104143877, 1123.0, 4.513392438930502, 763.0, 52.30227887677106, 1759.0, 153.0))","List(1, 22, List(), List(0.04201172203737228, 0.2908104650747991, 2.406151349554706, 3.7410799147356126, 13.889709455086422, 1.6446668758830796, 0.02769192535651848, 1.1135081655890402, 0.0, 12.380794389583158, 14.4005506282904, 0.06739504841282014, 3.6077195113948237, 0.23213294447012395, 1.4278845707177827, 19.6892871754614, 1.1222425658147295, 2.3106445950514747, 0.7348350936167161, 2.684018700538735, 2.3049935361982463, 0.10720426303996886))","List(1, 2, List(), List(4.343159113187271, -4.343159113187271))","List(1, 2, List(), List(0.9871713048257431, 0.012828695174256999))",0.0
39,2015-10-15T02:00:00.000+0000,0,1760.0,279.0,68.0,232.0,154.0,1844.0,44.0,6984.0,6984.0,1124.0,44.0,1484.0,764.0,100.29140877617509,11.284847828477886,442.628398602288,57.79405244919041,40.4627650363102,5.15593570715991,159.08171336934024,4.742598565323689,0,0,0,0,"List(1, 22, List(), List(44.0, 279.0, 4.742598565323689, 6984.0, 100.29140877617509, 11.284847828477886, 44.0, 1844.0, 0.0, 40.4627650363102, 442.628398602288, 68.0, 6984.0, 232.0, 1484.0, 159.08171336934024, 1124.0, 5.15593570715991, 764.0, 57.79405244919041, 1760.0, 154.0))","List(1, 22, List(), List(0.042988738828939076, 0.29185654588442067, 0.8113348212168924, 3.7416156558088955, 14.785459841903942, 2.881360741848478, 0.02833592362062356, 1.1141123479903363, 0.0, 12.755032069097572, 14.92374493964405, 0.06840094465778761, 3.6082361546013817, 0.23313784899163964, 1.4288474058969585, 18.830657541104745, 1.1232418913408335, 2.6395965197754863, 0.7357981802400669, 2.9658424237884766, 2.306303936161975, 0.10790494449774643))","List(1, 2, List(), List(4.540672972913085, -4.540672972913085))","List(1, 2, List(), List(0.9894463416492582, 0.010553658350741709))",0.0
39,2015-10-15T03:00:00.000+0000,0,1761.0,280.0,69.0,233.0,155.0,1845.0,45.0,6985.0,6985.0,1125.0,45.0,1485.0,765.0,100.4415699380936,11.343353478846133,442.92287647832774,57.404479717880825,38.9660110968309,6.077115734667256,165.1331519461085,13.650043784109032,0,0,0,0,"List(1, 22, List(), List(45.0, 280.0, 13.650043784109032, 6985.0, 100.4415699380936, 11.343353478846133, 45.0, 1845.0, 0.0, 38.9660110968309, 442.92287647832774, 69.0, 6985.0, 233.0, 1485.0, 165.1331519461085, 1125.0, 6.077115734667256, 765.0, 57.404479717880825, 1761.0, 155.0))","List(1, 22, List(), List(0.04396575562050587, 0.2929026266940422, 2.335166192255403, 3.742151396882179, 14.80759734955741, 2.896298992386688, 0.02897992188472864, 1.1147165303916327, 0.0, 12.283212002414666, 14.933673612829645, 0.06940684090275506, 3.6087527978079397, 0.23414275351315533, 1.4298102410761344, 19.546972226536816, 1.1242412168669373, 3.111197356713476, 0.7367612668634178, 2.945850551187314, 2.307614336125703, 0.108605625955524))","List(1, 2, List(), List(4.386323525789271, -4.386323525789271))","List(1, 2, List(), List(0.9877066044769235, 0.01229339552307655))",0.0
39,2015-10-15T04:00:00.000+0000,0,1762.0,281.0,70.0,234.0,156.0,1846.0,46.0,6986.0,6986.0,1126.0,46.0,1486.0,766.0,102.36820210530072,10.50187836582813,470.885704935064,58.13539678531709,38.02902267033148,5.222

In [26]:
# evaluate. note only 2 metrics are supported out of the box by Spark ML.
bce = (BinaryClassificationEvaluator()
               .setLabelCol("label")
               .setRawPredictionCol('rawPrediction'))
au_roc = bce.setMetricName('areaUnderROC').evaluate(df_pred)
au_prc = bce.setMetricName('areaUnderPR').evaluate(df_pred)

print("Area under ROC: {}".format(au_roc))
print("Area Under PR: {}".format(au_prc))

Area under ROC: 0.8226049714805794
Area Under PR: 0.1546244244518548

#Model Persistence

In [28]:
print(model_name[:-4])

PdM_logistic_regression

In [29]:
## NOTE: by default the model is saved to and loaded from /dbfs/ instead of cwd!
model_p_best.write().overwrite().save(model_name[:-4])
print("saved model to {}".format(model_dbfs))

saved model to /dbfs/PdM_logistic_regression.mml

In [30]:
%sh

ls -la /dbfs/PdM_logistic_regression/*

/dbfs/PdM_logistic_regression/metadata:
total 0
drwxr-xr-x 1 root root 0 Mar 4 18:15 .
drwxr-xr-x 1 root root 0 Mar 4 18:15 ..
-rw-r--r-- 1 root root 273 Mar 4 18:15 part-00000
-rw-r--r-- 1 root root 0 Mar 4 18:15 _SUCCESS

/dbfs/PdM_logistic_regression/stages:
total 0
drwxr-xr-x 1 root root 0 Mar 4 18:15 .
drwxr-xr-x 1 root root 0 Mar 4 18:15 ..
drwxr-xr-x 1 root root 0 Mar 4 18:15 0_VectorAssembler_c350b5d2e6a8
drwxr-xr-x 1 root root 0 Mar 4 18:15 1_StandardScaler_15dd431b4d4f
drwxr-xr-x 1 root root 0 Mar 4 18:15 2_LogisticRegression_c1bf9b390dcb

In [31]:
# You can ignore this code, we use it for testing our notebooks.
assert au_roc > .82

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.